In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from sklearn.svm import SVC, SVR
from sklearn.metrics import classification_report

In [12]:
EPS = 10e-6

def is_null(value):
  if abs(value) < EPS:
    return True
  else:
    return False

def get_class_num(fracture_posBlock, delta_posBlock, delta_rpmAv, delta_revVap, delta_flowInAv, delta_ndHole):
  if (delta_rpmAv > 5) & (delta_revVap > 5) & (delta_flowInAv > 0.5):
    if delta_ndHole < 0.1:
      if is_null(delta_posBlock):
        return 108
      else:
        return 107
    else:
      if is_null(delta_posBlock):
        return 10
      else:
        if delta_posBlock < 0:
          if (fracture_posBlock > 14) & (fracture_posBlock < 26):
            return 113
          if (fracture_posBlock > 8) & (fracture_posBlock < 14):
            return 114
        else:
          if (fracture_posBlock < 1) & (delta_posBlock < 0.1):
            return 124
          if (fracture_posBlock > 14) & (fracture_posBlock < 25):
            return 109
  if (delta_flowInAv < 4) & (delta_ndHole > 1.1):
    return 89
  if (delta_rpmAv < 5) & (delta_revVap < 5) & (delta_flowInAv > 0.5):
    if (delta_posBlock < 0) & (fracture_posBlock > 0) & (fracture_posBlock < 8):
      return 8
    if (delta_posBlock > 0) & (fracture_posBlock < 8):
      return 110
    if (delta_posBlock > 0.1) & (fracture_posBlock < 1):
      return 112
    if is_null(delta_posBlock):
      return 128
  if (delta_posBlock < 0):
    return 123
  if (delta_rpmAv < 5) & (delta_revVap < 5) & is_null(delta_posBlock):
    return 122
  if delta_posBlock > 0:
    if (fracture_posBlock > 14) & (fracture_posBlock < 26):
      return 115
    if (fracture_posBlock > 8) & (fracture_posBlock < 14):
      return 116
    if (fracture_posBlock > 0) & (fracture_posBlock < 8):
      return 9
    if (fracture_posBlock > 14) & (fracture_posBlock < 26):
      return 117
  return -1

In [13]:
fracture_posBlock = np.array([np.random.normal(15, 15, 50000)]).T
delta_posBlock = np.array([np.random.normal(0, 2, 50000)]).T
delta_rpmAv = np.array([np.random.normal(5, 5, 50000)]).T
delta_revVap = np.array([np.random.normal(5, 5, 50000)]).T
delta_flowInAv = np.array([np.random.normal(0.5, 7, 50000)]).T
delta_ndHole = np.array([np.random.normal(1, 3, 50000)]).T

In [14]:
data = np.hstack((fracture_posBlock,delta_posBlock,delta_rpmAv,delta_revVap,delta_flowInAv,delta_ndHole))
X = pd.DataFrame(data, columns=['fracture_posBlock', 'delta_posBlock', 'delta_rpmAv', 'delta_revVap', 'delta_flowInAv', 'delta_ndHole'])
y = np.array([], dtype=int)

In [15]:
for args in X.values:
  y = np.append(y, [get_class_num(args[0],args[1],args[2],args[3],args[4],args[5])])

In [16]:
X = X[y != -1]
y = y[y != -1]

In [17]:
unique, counts = np.unique(y, return_counts=True)
np.asarray((unique, counts)).T

array([[    8,   401],
       [    9,  2057],
       [   89, 16070],
       [  107,  2510],
       [  109,   533],
       [  110,   784],
       [  113,   540],
       [  114,   285],
       [  115,  4091],
       [  116,  2374],
       [  123, 14662],
       [  124,    21]])

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [19]:
model = SVC(kernel='linear', verbose=1)
_ = model.fit(X_train, Y_train)

[LibSVM]

In [20]:
model.score(X_test, Y_test)

0.7981051206857659